# Optimizing fertilizer purchase plan and cost

In the following code, we will implement the fertilizer purchase and cost problem in Pyomo. The model is given as follows:

**Decision variables**

$A$ = pounds to fertilizer A to be purchased

$B$ = pounds to fertilizer B to be purchased

$C$ = pounds to fertilizer C to be purchased

**Objective function**

Minimize total cost
$$
z = 0.01 A + 0.008 B + 0.007 C
$$

**Constraints**

Subject to the following mineral content requirements/constraints

$$
\begin{align}
Nitrogen\\
0.02  A   & + & 0.01  B   & + & 0.015 C  & \ge 1.2  & \text{(lb)}\\
Phosphorus\\
0.01  A   & + & 0.005 B   & + & 0.01  C  & \ge 1.4  & \text{(lb)}\\
Potash\\
0.005 A   & + & 0.015 B   & + & 0.005 C  & \ge 1.8  & \text{(lb)}
\end{align}
$$

**Install Pyomo and a few optimization engines**

In [ ]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre    # Installs a set of good open source optimization tools
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'    # Add to path so we can find it

import pyomo.environ as pyo         # Import pyomo

**Instantiate an engine that we can use to optimize**

In [ ]:
opt = pyo.SolverFactory('cbc')

**The next step is to create an optimization model in Pyomo**

In [ ]:
m = pyo.ConcreteModel()
  # Instantiate a model

# Define variables
m.A = pyo.Var(domain=pyo.NonNegativeReals)
m.B = pyo.Var(domain=pyo.NonNegativeReals)
m.C = pyo.Var(domain=pyo.NonNegativeReals)
  # three variables, defined to be continuous and non-negative.

m.TotalCost = pyo.Objective( expr = 0.01*m.A + 0.008*m.B + 0.007*m.C, sense=pyo.minimize )

m.NitrogenContent = pyo.Constraint( expr = 0.02*m.A + 0.01*m.B + 0.015*m.C >= 1.2 )
m.PhosphorusContent = pyo.Constraint( expr = 0.01*m.A + 0.005*m.B + 0.01*m.C >= 1.4 )
m.PotashContent = pyo.Constraint( expr = 0.005*m.A + 0.015*m.B + 0.005*m.C >= 1.8 )

In [ ]:
# We can print the model to get a sense of what is in it
m.pprint()

3 Var Declarations
    A : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    B : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    C : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    TotalCost : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 0.01*A + 0.008*B + 0.007*C

3 Constraint Declarations
    NitrogenContent : Size=1, Index=None, Active=True
        Key  : Lower : Body                      : Upper : Active
        None :   1.2 : 0.02*A + 0.01*B + 0.015*C :  +Inf :   True
    PhosphorusContent : Size=1, Index=None, Active=True
        Key  : Lower : Body                    

In [ ]:
m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
  # Only necessary to access the shadow price here. Must be done before optimizing.
  # The Suffix is the mechanism through which Pyomo accesses the internal variables.
  # IMPORT here suggests that we are going to use the variable to receive some information from the
  # optimizer (in contrast to send some extra information to it).

opt.solve(m)
print("Decision variables: ")
print("Fertilizer A  ", m.A.value)
print("Fertilizer B  ", m.B.value)
print("Fertilizer C  ", m.C.value)

print("\nObjective: ")
print("Total cost obtained", m.TotalCost())

# We can also get the LHS of the constraints, showing us how much of the resources were used:
print("\nResources used: ")
print("Nitrogen content   ", m.NitrogenContent())
print("Phosphorus content   ", m.PhosphorusContent())
print("Potash content   ", m.PotashContent())

Decision variables: 
Fertilizer A   0.0
Fertilizer B   88.0
Fertilizer C   96.0

Objective: 
Total cost obtained 1.376

Resources used: 
Nitrogen content    2.32
Phosphorus content    1.4
Potash content    1.7999999999999998
